# Projeto Final | Sistema de Controle Financeiro

%----------------------------------------------------------------------------%<br>
<br>
``Autores:`` Andrea Elias, Anthony Heimlich, Éverton Donato, Julia Midori e Luana Kruger  <br>
<br>
``Instituição:`` ADA Tech<br>
<br>
``Projeto:`` Santander Coders 2023.2<br>
<br>
``Descrição:`` Este código desenvolve um sistema para controle financeiro que receba as movimentações e as armazena em um arquivo csv ou json.<br>
<br>
``Repositório GitHub:`` https://github.com/JuliaMidoriRW/Trabalho_Final_LogProgII_Grupo4 <br>
<br>

%----------------------------------------------------------------------------%

# Carregar bibliotecas e lista de registros

In [30]:
from datetime import datetime
from dateutil import relativedelta
import os

In [31]:
registros = []

# Carregar funções

### Função Calcular rendimento

In [32]:
def calcula_rendimento(valor, data):
    """
    Calcular montante com base no valor e na data do registro.

    Parameters
    ----------
    valor: float
        Valor do registro financeiro.
        
    data: datetime
        Data do registro no formato 'YYYY-MM-DD'.

    Returns
    -------
    montante: float
        Valor do rendimento considerando uma taxa de rendimento diário.

    """
    
    data_registro = datetime.strptime(data, "%Y-%m-%d").date()
    data_agora = datetime.now().date()
    delta = relativedelta.relativedelta(data_agora, data_registro)

    meses = delta.months + (delta.years * 12)
    taxa = 0.01  # Exemplo: taxa de rendimento diária (1%)
    montante = (valor * ((1 + taxa) ** meses)) - valor

    return montante

### Função Criar registro

In [33]:
def criar_registro(tipo, valor, data):
    """
    Criar novos registros e identificar a data que o registro foi feito, qual tipo de movimentação, valor.

    Parameters
    ----------
    tipo: string
        Tipo da movimentação realizada. Os tipos podem ser:
        - Receita (valor numérico e armazenado normalmente), 
        - Despesas (valor positivo, mas armazenado como negativo),
        - Investimento (tem a informação de 'Montante').

    valor: float
        Valor do registro.

    data: datetime
        Data do registro no formato 'YYYY-MM-DD'.

    Returns
    -------
    None

    Exemplo
    -------
    >>> criar_registro("Receita", 100, "2022-01-01")
    >>> criar_registro("Despesa", 50, "2022-01-02")
    >>> criar_registro("Investimento", 200, "2022-01-03")

    """

    data_registro = datetime.strptime(data, "%Y-%m-%d").date()
    dia, mes, ano = data_registro.day, data_registro.month, data_registro.year
    
    registro = {
        'tipo': tipo.lower()
        , 'valor': -valor if tipo.lower() == 'despesa' else valor
        , 'dia': dia
        , 'mes': mes
        , 'ano': ano
    }

    if tipo.lower() == 'investimento':
        registro['montante'] = calcula_rendimento(valor, data)
        
    registros.append(registro)

### Função Ler registro

In [47]:
def ler_registros(chave, valor):
    """
    Consultar os registros por data, tipo ou valor.

    Parameters
    ----------
    chave: string
        Chave pela qual os registros serão consultados ('data', 'tipo' ou 'valor').

    valor: string or float
        Valor associado à chave para filtrar os registros.

    Returns
    -------
    resultado: list[dict]
        Lista de registros que correspondem aos critérios de consulta.


    Exemplo
    -------
    >>> ler_registros('tipo', 'Receita')
    >>> ler_registros('data', '2022-02-15')
    >>> ler_registros('valor', 1000)

    """

    resultado = []

    valor = valor.lower() if isinstance(valor,str) else valor
    chave = chave.lower()

    for operacao in registros:
        if chave != "data":
            if operacao[chave] == valor:
                resultado.append(operacao)
        else:
            data_registro = datetime.strptime(valor, "%Y-%m-%d").date()
            dia, mes, ano = data_registro.day, data_registro.month, data_registro.year
            if operacao["dia"] == dia and operacao["mes"] == mes and operacao["ano"] == ano:
                resultado.append(operacao)

    return resultado


### Função Atualizar registro

In [49]:
def atualizar_registro(indice, tipo, valor):
    """
    Atualiza o valor e/ou o tipo do registro no índice especificado.

    Parameters
    ----------
    indice: int
        Índice do registro a ser atualizado.

    tipo: string
        Tipo da movimentação financeira ('Receita', 'Despesa' ou 'Investimento').

    valor: float
        Valor da movimentação financeira.

    Returns
    -------
    None

    Nota
    -------
    - Se o índice estiver fora dos limites dos registros existentes, nenhum registro será atualizado.
    - A data do registro é automaticamente atualizada para a data e hora atuais.
    - Se o tipo for especificado como 'Investimento', o montante também será recalculado.

    Exemplo
    -------
    >>> atualizar_registro(0, tipo='Receita', valor=1500.0)
    >>> atualizar_registro(2, tipo='Investimento', valor=100)
    
    """
    
    tipo = tipo.lower()
    valor = valor.lower() if isinstance(valor,str) else valor
    
    if indice < len(registros):
        registro = registros[indice]
        if tipo:
            registro['tipo'] = tipo
        if valor:
            registro['valor'] = valor
            
        data_agora = datetime.now()
        dia, mes, ano = data_agora.day, data_agora.month, data_agora.year
        registro['dia'], registro['mes'], registro['ano'] = dia, mes, ano

        if registro['tipo'] == "investimento":
            registro['montante'] = calcula_rendimento(registro['valor'], f"{ano}-{mes:02d}-{dia:02d}")

        print('Registro', indice, 'atualizado com sucesso')
    
    else:
        print('Índice fora dos limites dos registros existentes, nenhum registro foi atualizado.')

### Função Deletar registro

In [36]:
def deletar_registro(indice):
    """
    Deleta o registro no índice especificado.

    Parameters
    ----------
    indice: int
        Indice do registro a ser deletado.

    Returns
    -------
    None

    Nota
    -------
    - Se o índice estiver fora dos limites dos registros existentes, nenhum registro será deletado.

    Exemplo
    -------
    >>> deletar_registro(0)
    >>> deletar_registro(2)

    """
    if indice < len(registros):
        del registros[indice]
        print('Registro', indice, 'deletado com sucesso')
    
    else:
        print('Índice fora dos limites dos registros existentes, nenhum registro foi deletado.')

### Função Atualizar rendimento

In [37]:
def atualiza_rendimento():
    """
    Atualiza os valores de rendimento para todas as movimentações do tipo 'Investimento'.

    Parameters
    ----------
    None

    Returns
    -------
    None

    Nota
    -------
    - Atualiza os montantes para todas as movimentações do tipo 'Investimento' com base na taxa de rendimento diária.
    - A data do investimento é obtida a partir dos campos 'dia', 'mes' e 'ano' de cada movimentação.

    Exemplo
    -------
    >>> atualiza_rendimento()

    """
    
    for movimento in registros:
        if movimento["tipo"] == "investimento":
            valor_inicial = movimento["valor"]
            concatenar = str(movimento['ano']) + '-' + str(movimento['mes']) + '-' + str(movimento['dia'])
            data_investimento = datetime.strptime(concatenar, "%Y-%m-%d")
            movimento["montante"] = calcula_rendimento(valor_inicial, data_investimento.strftime("%Y-%m-%d"))

### Função Exportar relatório

In [38]:
def exportar_relatorio():
    """
    Exporta um relatório final em formato CSV.

    Parameters
    ----------
    None

    Returns
    -------
    None

    Nota
    -------
    - O relatório é exportado para um arquivo com nome padrão 'relatorio.csv'.
    - O relatório inclui informações sobre tipo, valor, data e montante para cada movimentação no formato especificado.

    Exemplo:
    >>> exportar_relatorio()

    """

    nome_arquivo = "relatorio.csv"

    with open(nome_arquivo, "w") as file:
        file.write("Tipo, Valor, Data, Montante\n")
        for movimento in registros:
            tipo = movimento.get('tipo', '')
            valor = movimento.get('valor', '')
            dia = movimento.get('dia', '')
            mes = movimento.get('mes', '')
            ano = movimento.get('ano', '')
            montante = movimento.get('montante', 0)


            data_formatada = f"{ano}-{mes:02d}-{dia:02d}"

            file.write(f"{tipo},{valor},{data_formatada},{montante}\n")

    print(f"Relatório exportado para {nome_arquivo}")


### Função Agrupar 

In [66]:
def agrupar_por(chave):
    """
    Função de agrupamento capaz de mostrar o total de valor baseado em alguma informação (tipo, mes, ano, data).

    Parameters
    ----------
    chave: string
        Informação base para o agrupamento ('tipo', 'mes', 'ano', 'data')

    Returns
    -------
    resultado: dict
        Dicionário com o registro agrupado por chave

    """
        
    resultado = {}
    for operacao in registros:
        valor = operacao["valor"]
        if chave == "tipo":
            chave_valor = operacao["tipo"]
        elif chave == "mes":
            chave_valor = str(operacao["ano"]) + "-" + str(operacao["mes"]).zfill(2)
        elif chave == "ano":
            chave_valor = operacao["ano"]
        else:
            chave_valor = str(operacao["ano"]) + "-" + str(operacao["mes"]).zfill(2) + "-" + str(operacao["dia"]).zfill(2)

        resultado[chave_valor] = resultado.get(chave_valor, 0) + valor

    return dict(sorted(resultado.items()))

### Função Verificar formato data


In [40]:
def validarFormatoData(data):
    """ 
    Função utilizada para verificar se a data está no formato desejado YYYY-MM-DD

    Parameters
    ----------
    data: string
        Recebe a data que o usuário inseriu 

    Returns
    -------
    Boolean: True / False

    Nota
    -------
    - Após separar a data é verificado se o ano inserido possui 4 caracteres, se o mês inserido está entre 1 e 12 e se o dia inserido está entre 1 e 31

    """
    
    try:
        ano, mes, dia = map(int, data.split('-'))
        if len(str(ano)) == 4 and 1 <= mes <= 12 and 1 <= dia <= 31:
            return True
    except ValueError:
         print('Formato de data inválido!')
    return False

# Testes executados

### Criar registro

In [41]:
criar_registro("Receita", 100, "2022-01-01")
criar_registro("Despesa", 50, "2022-01-02")
criar_registro("Investimento", 200, "2022-01-03")
criar_registro("Receita", 1000, "2022-01-01")
criar_registro("Despesa", 90, "2022-01-02")
criar_registro("Investimento", 200, "2023-05-03")
criar_registro("Receita", 550, "2023-02-03")
criar_registro("despesa", 230, "2023-02-03")

registros

[{'tipo': 'receita', 'valor': 100, 'dia': 1, 'mes': 1, 'ano': 2022},
 {'tipo': 'despesa', 'valor': -50, 'dia': 2, 'mes': 1, 'ano': 2022},
 {'tipo': 'investimento',
  'valor': 200,
  'dia': 3,
  'mes': 1,
  'ano': 2022,
  'montante': 53.946929706382946},
 {'tipo': 'receita', 'valor': 1000, 'dia': 1, 'mes': 1, 'ano': 2022},
 {'tipo': 'despesa', 'valor': -90, 'dia': 2, 'mes': 1, 'ano': 2022},
 {'tipo': 'investimento',
  'valor': 200,
  'dia': 3,
  'mes': 5,
  'ano': 2023,
  'montante': 16.571341125616044},
 {'tipo': 'receita', 'valor': 550, 'dia': 3, 'mes': 2, 'ano': 2023}]

### Ler registro

In [48]:
print(ler_registros('tipo', 'Receita'))

print(ler_registros('data', '2022-01-03'))

print(ler_registros('valor', 1000))

[{'tipo': 'receita', 'valor': 100, 'dia': 1, 'mes': 1, 'ano': 2022}, {'tipo': 'receita', 'valor': 1000, 'dia': 1, 'mes': 1, 'ano': 2022}, {'tipo': 'receita', 'valor': 550, 'dia': 3, 'mes': 2, 'ano': 2023}]
[{'tipo': 'investimento', 'valor': 200, 'dia': 3, 'mes': 1, 'ano': 2022, 'montante': 53.946929706382946}]
[{'tipo': 'receita', 'valor': 1000, 'dia': 1, 'mes': 1, 'ano': 2022}]


### Atualizar registro

In [51]:
print(registros[0])
atualizar_registro(0, tipo='Receita', valor=1500.0)
print(registros[0])

{'tipo': 'receita', 'valor': 1500.0, 'dia': 24, 'mes': 1, 'ano': 2024}
Registro 0 atualizado com sucesso
{'tipo': 'receita', 'valor': 1500.0, 'dia': 24, 'mes': 1, 'ano': 2024}


In [52]:
# Verificar a atualização do montante
print(registros[2])
atualizar_registro(2, tipo='Investimento', valor=100)
print(registros[2])

{'tipo': 'investimento', 'valor': 200, 'dia': 3, 'mes': 1, 'ano': 2022, 'montante': 53.946929706382946}
Registro 2 atualizado com sucesso
{'tipo': 'investimento', 'valor': 100, 'dia': 24, 'mes': 1, 'ano': 2024, 'montante': 0.0}


### Deletar registro

In [53]:
print(registros[1])
deletar_registro(1)
print(registros[1])

{'tipo': 'despesa', 'valor': -50, 'dia': 2, 'mes': 1, 'ano': 2022}
Registro 1 deletado com sucesso
{'tipo': 'investimento', 'valor': 100, 'dia': 24, 'mes': 1, 'ano': 2024, 'montante': 0.0}


### Atualizar rendimento

In [54]:
criar_registro("Investimento", 500, "2023-01-03")
print(ler_registros('tipo', 'Investimento'))
atualiza_rendimento()
print(ler_registros('tipo', 'Investimento'))

[{'tipo': 'investimento', 'valor': 100, 'dia': 24, 'mes': 1, 'ano': 2024, 'montante': 0.0}, {'tipo': 'investimento', 'valor': 200, 'dia': 3, 'mes': 5, 'ano': 2023, 'montante': 16.571341125616044}, {'tipo': 'investimento', 'valor': 500, 'dia': 3, 'mes': 1, 'ano': 2023, 'montante': 63.41251506598485}]
[{'tipo': 'investimento', 'valor': 100, 'dia': 24, 'mes': 1, 'ano': 2024, 'montante': 0.0}, {'tipo': 'investimento', 'valor': 200, 'dia': 3, 'mes': 5, 'ano': 2023, 'montante': 16.571341125616044}, {'tipo': 'investimento', 'valor': 500, 'dia': 3, 'mes': 1, 'ano': 2023, 'montante': 63.41251506598485}]


### Exportar relatorio .CSV

In [69]:
exportar_relatorio()

Relatório exportado para relatorio.csv


### Agrupar por tipo

In [67]:
print("Agrupado por tipo:", agrupar_por("tipo"))
print("Agrupado por ano:", agrupar_por("ano"))
print("Agrupado por mês:", agrupar_por("mes"))
print("Agrupado por data:", agrupar_por("data"))

Agrupado por tipo: {'despesa': -320, 'investimento': 800, 'receita': 3050.0}
Agrupado por mês: {'2022-01': 910, '2023-01': 500, '2023-02': 320, '2023-05': 200, '2024-01': 1600.0}
Agrupado por ano: {2022: 910, 2023: 1020, 2024: 1600.0}
Agrupado por data: {'2022-01-01': 1000, '2022-01-02': -90, '2023-01-03': 500, '2023-02-03': 320, '2023-05-03': 200, '2024-01-24': 1600.0}


### Consultas por tipo

In [70]:
print("Pesquisa por tipo Receita:", ler_registros("tipo",'Receita'))
print("Pesquisa por tipo Despesa:", ler_registros("tipo",'Despesa'))
print("Pesquisa por tipo Investimento:", ler_registros("tipo",'Investimento'))

print("Pesquisa por valor:", ler_registros("valor",100))
print("Pesquisa por valor:", ler_registros("valor",-50))

print("Pesquisa por data:", ler_registros("data","2022-01-01"))
print("Pesquisa por data:", ler_registros("data","2024-01-24"))

Pesquisa por tipo Receita: [{'tipo': 'receita', 'valor': 1500.0, 'dia': 24, 'mes': 1, 'ano': 2024}, {'tipo': 'receita', 'valor': 1000, 'dia': 1, 'mes': 1, 'ano': 2022}, {'tipo': 'receita', 'valor': 550, 'dia': 3, 'mes': 2, 'ano': 2023}]
Pesquisa por tipo Despesa: [{'tipo': 'despesa', 'valor': -90, 'dia': 2, 'mes': 1, 'ano': 2022}, {'tipo': 'despesa', 'valor': -230, 'dia': 3, 'mes': 2, 'ano': 2023}]
Pesquisa por tipo Investimento: [{'tipo': 'investimento', 'valor': 100, 'dia': 24, 'mes': 1, 'ano': 2024, 'montante': 0.0}, {'tipo': 'investimento', 'valor': 200, 'dia': 3, 'mes': 5, 'ano': 2023, 'montante': 16.571341125616044}, {'tipo': 'investimento', 'valor': 500, 'dia': 3, 'mes': 1, 'ano': 2023, 'montante': 63.41251506598485}]
Pesquisa por valor: [{'tipo': 'investimento', 'valor': 100, 'dia': 24, 'mes': 1, 'ano': 2024, 'montante': 0.0}]
Pesquisa por valor: []
Pesquisa por data: [{'tipo': 'receita', 'valor': 1000, 'dia': 1, 'mes': 1, 'ano': 2022}]
Pesquisa por data: [{'tipo': 'receita', '

# Testes com interação com o usuário

In [28]:
escolha = "0"
loop_check = True

while escolha != "8":
    print("\n----- Menu -----")
    print("1. Criar Registro")
    print("2. Ler Registros")
    print("3. Atualizar Registro")
    print("4. Deletar Registro")
    print("5. Atualizar Rendimento")
    print("6. Exportar Relatório")
    print("7. Agrupar Registros")
    print("8. Sair")

    escolha = input("Escolha uma opção (1-8):")

    if escolha == '1':
        loop_check = True
        while loop_check:
            tipo = input("Tipo (Receita/Despesa/Investimento): ").lower()
            if tipo in ['receita', 'despesa', 'investimento']:
                loop_check = False
            else:
                print("Tipo inválido! Insira um tipo válido")

        loop_check = True
        while loop_check:
            try:
                valor = float(input("Valor: "))
                loop_check = False
            except ValueError:
                print("Valor inválido! Insira um valor válido")

        loop_check = True
        while loop_check:
            data = input("Data (YYYY-MM-DD): ")
            if validarFormatoData(data):
                loop_check = False
            else:
                print("Formato de data inválido! Digite a data no formato correto! (YYYY-MM-DD)")

        criar_registro(tipo, valor, data)
        print("Registro criado com sucesso!")

    elif escolha == '2':
        loop_check = True
        while loop_check:
            chave = input("Chave para consulta (tipo/valor/data): ").lower()
            if chave in ['tipo','valor','data']:
                loop_check = False
            else:
                print("Chave inválida! Insira uma chave válida")

        loop_check = True
        while loop_check:
            try:
                valor = input("Valor para consulta: ")
                loop_check = False
            except ValueError:
                print("Valor inválido! Insira um valor válido")

        resultado = ler_registros(chave, valor)
        print("Resultados:")
        for movimento in resultado:
            print(movimento)

    elif escolha == '3':
        loop_check = True
        while loop_check:
            try:
                indice = int(input("Índice do registro a ser atualizado: "))
                loop_check = False
            except ValueError:
                print("Índice inválido! Insira um índice válido")

        loop_check = True     
        while loop_check:
            tipo = input("Tipo (Receita/Despesa/Investimento): ").capitalize()
            if tipo in ['Receita', 'Despesa', 'Investimento']:
                loop_check = False
            else:
                print("Tipo inválido! Insira um tipo válido")

        loop_check = True
        while loop_check:
            try:
                valor = float(input("Valor: "))
                loop_check = False
            except ValueError:
                print("Valor inválido! Insira um valor válido")

        atualizar_registro(indice, tipo, valor)
        print("Registro atualizado com sucesso!")

    elif escolha == '4':
        loop_check = True
        while loop_check:
            try:
                indice = int(input("Índice do registro a ser deletado: "))
                loop_check = False
            except ValueError:
                print("Índice inválido! Insira um índice válido")
                
        deletar_registro(indice)

    elif escolha == '5':
        atualiza_rendimento()
        print("Rendimento atualizado para todos os investimentos.")

    elif escolha == '6':
        print("Seu relatório será gerado em .csv")
        exportar_relatorio()

    elif escolha == '7':
        loop_check = True
        while loop_check:
            chave = input("Chave para agrupamento (tipo/mes/ano/data): ").lower()
            if chave in ['tipo','mes', 'ano', 'data']:
                loop_check = False
            else:
                print("Chave inválida! Insira uma chave válida")
        resultado = agrupar_por(chave)
        print("Resultados:")
        print(resultado)

    elif escolha == '8':
        print("Saindo do programa. Até mais!")
        loop_check = False
    else:
        print("Opção inválida. Por favor, escolha uma opção válida.")


----- Menu -----
1. Criar Registro
2. Ler Registros
3. Atualizar Registro
4. Deletar Registro
5. Atualizar Rendimento
6. Exportar Relatório
7. Agrupar Registros
8. Sair
Tipo inválido! Insira um tipo válido
Valor inválido! Insira um valor válido
Valor inválido! Insira um valor válido
Formato de data inválido!
Formato de data inválido! Digite a data no formato correto! (YYYY-MM-DD)
Formato de data inválido!
Formato de data inválido! Digite a data no formato correto! (YYYY-MM-DD)
Formato de data inválido!
Formato de data inválido! Digite a data no formato correto! (YYYY-MM-DD)
Formato de data inválido! Digite a data no formato correto! (YYYY-MM-DD)
Registro criado com sucesso!

----- Menu -----
1. Criar Registro
2. Ler Registros
3. Atualizar Registro
4. Deletar Registro
5. Atualizar Rendimento
6. Exportar Relatório
7. Agrupar Registros
8. Sair
Chave inválida! Insira uma chave válida
Resultados:
{'tipo': 'receita', 'valor': 300.0, 'dia': 6, 'mes': 4, 'ano': 2023}

----- Menu -----
1. Cria